<a href="https://colab.research.google.com/github/Htets-Corner/SYNTHBUSTER_RAISE-1k/blob/main/syn_real_mobilevit050.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Step 0: Data Preparation

# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Define dataset path
import os

# Update this path if needed
dataset_path = "/content/drive/MyDrive/Dataset"

train_dir = os.path.join(dataset_path, "train")
test_dir  = os.path.join(dataset_path, "test")

print("Train path:", train_dir)
print("Test path:", test_dir)

# 3. Import necessary libraries
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 4. Define image transformations (resize, normalization)
# MobileViT usually works with 256x256 or 224x224 input
image_size = 256

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],   # normalize to [-1, 1]
        std=[0.5, 0.5, 0.5]
    )
])

# 5. Load train and test datasets
#train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
#test_dataset  = datasets.ImageFolder(root=test_dir, transform=transform)

from torchvision.datasets import ImageFolder

valid_exts = ('.jpg', '.jpeg', '.png', '.PNG', '.bmp', '.tif', '.tiff', '.webp')

train_dataset = ImageFolder(
    root=train_dir,
    transform=transform,
    is_valid_file=lambda path: path.lower().endswith(valid_exts)
)

test_dataset = ImageFolder(
    root=test_dir,
    transform=transform,
    is_valid_file=lambda path: path.lower().endswith(valid_exts)
)


# 6. Create DataLoaders
#batch_size = 32
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 7. Check class labels
classes = train_dataset.classes
print("Classes:", classes)
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train path: /content/drive/MyDrive/Dataset/train
Test path: /content/drive/MyDrive/Dataset/test
Classes: ['ai', 'real']
Train size: 3199
Test size: 800


In [ ]:
# @title
import os

print("Train/AI files:", len(os.listdir(train_dir + "/ai")))
print("Train/Real files:", len(os.listdir(train_dir + "/real")))
print("Test/AI files:", len(os.listdir(test_dir + "/ai")))
print("Test/Real files:", len(os.listdir(test_dir + "/real")))

# Show first 5 files in each folder
print("Sample AI:", os.listdir(train_dir + "/ai")[:5])
print("Sample Real:", os.listdir(train_dir + "/real")[:5])


Train/AI files: 2400
Train/Real files: 799
Test/AI files: 600
Test/Real files: 200
Sample AI: ['stable-diffusion-2_r179bb406t.png', 'stable-diffusion-2_r0e965ba7t.png', 'firefly_r03f70ccdt.png', 'stable-diffusion-xl_r164c1e13t.png', 'firefly_r138ad247t.png']
Sample Real: ['r15c91802t.png', 'r0db6cc31t.png', 'r1aa53167t.png', 'r05d9d749t.png', 'r13de1e12t.png']


In [ ]:
# @title
# Run this first cell in Colab
!pip install -q timm        # recommended: timm includes MobileViT variants + pretrained weights
# optional fallback (if you want the standalone implementation)
# !pip install -q git+https://github.com/chinhsuanwu/mobilevit-pytorch.git


In [ ]:
# @title
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
import timm
from collections import Counter
print("Torch:", torch.__version__, "Timm:", timm.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Torch: 2.8.0+cu126 Timm: 1.0.19
Device: cpu


In [ ]:
# @title
# list available MobileViT-like models in timm
models = timm.list_models('*mobilevit*')
print("MobileViT variants found in timm:", models)

# choose one automatically (you can change this name)
if len(models) == 0:
    raise RuntimeError("No MobileViT models found in timm. You can pip-install a standalone MobileViT repo instead.")
model_name = models[0]   # default to first found; or set e.g. 'mobilevit_xxs' or 'mobilevit_s'
print("Using model:", model_name)



MobileViT variants found in timm: ['mobilevit_s', 'mobilevit_xs', 'mobilevit_xxs', 'mobilevitv2_050', 'mobilevitv2_075', 'mobilevitv2_100', 'mobilevitv2_125', 'mobilevitv2_150', 'mobilevitv2_175', 'mobilevitv2_200']


In [ ]:
# @title
import timm
import torch
import torch.nn as nn

# 1. Choose model
model_name = "mobilevitv2_050"

# 2. Load pretrained model from timm
model = timm.create_model(model_name, pretrained=True)

# 3. Adapt classifier head for 2 classes (ai, real)
if hasattr(model, "reset_classifier"):
    model.reset_classifier(num_classes=2)
else:
    # fallback if reset_classifier not available
    in_features = model.classifier.in_features
    model.classifier = nn.Linear(in_features, 2)

# 4. Move to device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"✅ Loaded {model_name} with classifier reset to 2 classes")


✅ Loaded mobilevitv2_050 with classifier reset to 2 classes


In [ ]:
# @title
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from collections import Counter
from tqdm import tqdm
import copy
import os

# -------------------------------
# 1) Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# -------------------------------
# 2) Dataset & DataLoader (reuse your train_dataset and test_dataset)
# Handle class imbalance with WeightedRandomSampler
targets = [s[1] for s in train_dataset.samples]
counts = Counter(targets)
class_weights = {cls: 1.0 / count for cls, count in counts.items()}
sample_weights = [class_weights[t] for t in targets]

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# -------------------------------
# 3) Loss, optimizer, scheduler
criterion = nn.CrossEntropyLoss()  # class indices 0/1
optimizer = optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)  # adjust T_max for epochs

# Mixed precision
scaler = torch.cuda.amp.GradScaler() if torch.cuda.is_available() else None

# -------------------------------
# 4) Train & Validation functions
def validate(model, loader):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
    return running_loss/total, correct/total

def train_one_epoch(model, loader, optimizer, scaler=None):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    loop = tqdm(loader, desc="Train")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        if scaler is not None:
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += images.size(0)
        loop.set_postfix(loss=running_loss/total, acc=100*correct/total)
    return running_loss/total, correct/total

# -------------------------------
# 5) Training loop with metrics storage
num_epochs = 10
best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())
save_path = "/content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth"

# Lists to store metrics per epoch
train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, scaler)
    # Validate
    val_loss, val_acc = validate(model, test_loader)
    # Scheduler step
    scheduler.step()

    # Store metrics
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss {train_loss:.4f} Acc {train_acc:.4f} | Val Loss {val_loss:.4f} Acc {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save({
            'epoch': epoch+1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': best_acc
        }, save_path)
        print(f"💾 Best model saved: {save_path}")

# Load best weights at the end
model.load_state_dict(best_model_wts)
print("✅ Training complete, best model loaded.")


Device: cpu


Train:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Train: 100%|██████████| 100/100 [35:24<00:00, 21.24s/it, acc=75.2, loss=0.552]


Epoch 1/10: Train Loss 0.5524 Acc 0.7518 | Val Loss 0.2921 Acc 0.8888
💾 Best model saved: /content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth


Train: 100%|██████████| 100/100 [34:20<00:00, 20.61s/it, acc=92.8, loss=0.19]


Epoch 2/10: Train Loss 0.1897 Acc 0.9281 | Val Loss 0.1341 Acc 0.9437
💾 Best model saved: /content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth


...

Epoch 6/10: Train Loss 0.0398 Acc 0.9869 | Val Loss 0.1091 Acc 0.9587
💾 Best model saved: /content/drive/MyDrive/Dataset/mobilevitv2_050_best.pth


Train: 100%|██████████| 100/100 [35:51<00:00, 21.52s/it, acc=98.5, loss=0.0437]


Epoch 7/10: Train Loss 0.0437 Acc 0.9853 | Val Loss 0.1188 Acc 0.9575


Train:   0%|          | 0/100 [00:50<?, ?it/s]


KeyboardInterrupt: 

In [1]:
# @title (RUN) Mount Drive and inspect dataset, list generators
# Colab cell 1: mount drive and show dataset structure
from pathlib import Path
import random, shutil, os
from collections import defaultdict

from google.colab import drive
drive.mount('/content/drive')

ROOT = Path("/content/drive/MyDrive/synthbuster_dataset")  # <-- change if different
if not ROOT.exists():
    raise RuntimeError(f"Path not found: {ROOT}")

print("Root:", ROOT)
children = [p.name for p in sorted(ROOT.iterdir()) if p.is_dir()]
print("Found folders:", children)

# Expecting: raise (real), stable-diffusion-xl, dalle3, firefly (names may vary)
ai_generators = [p.name for p in ROOT.iterdir() if p.is_dir() and p.name.lower() != "raise"]
real_folder = ROOT / "raise"
print("AI generators detected:", ai_generators)
print("Real folder:", real_folder.exists(), real_folder)


Mounted at /content/drive
Root: /content/drive/MyDrive/synthbuster_dataset
Found folders: ['dalle3', 'firefly', 'raise', 'stable-diffusion-xl']
AI generators detected: ['dalle3', 'firefly', 'stable-diffusion-xl']
Real folder: True /content/drive/MyDrive/synthbuster_dataset/raise


In [2]:
# @title (RUN) Utilities: valid ext, preprocessing & attack functions
# Colab cell 2: utilities (image ext, laplacian preprocessing, attack functions)
from PIL import Image, ImageFilter, ImageOps
import numpy as np
import io, random
import cv2
from pathlib import Path

valid_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp', '.PNG')

def is_image_file(p: Path):
    return p.suffix.lower() in valid_exts

# Optional Laplacian enhancement: apply and blend with original to emphasize high-frequency details
def apply_laplacian_pil(pil_img, blend=0.5):
    # pil_img: RGB PIL.Image
    np_im = np.array(pil_img.convert("L"))  # grayscale
    lap = cv2.Laplacian(np_im, cv2.CV_64F)
    lap = np.clip(np.abs(lap), 0, 255).astype(np.uint8)
    lap = Image.fromarray(lap).convert("RGB")
    # blend with original
    blended = Image.blend(pil_img, lap, alpha=blend)
    return blended

# Attack functions: inputs PIL.Image -> output PIL.Image
def attack_crop(pil_img, min_frac=0.05, max_frac=0.20):
    w, h = pil_img.size
    frac = random.uniform(min_frac, max_frac)
    area = w * h
    remove_area = int(area * frac)
    # remove a random rectangle of approx remove_area area (keep aspect ratio near original)
    rw = int((remove_area / h)**0.5 * w/h * h) if h>0 else int(w*0.1)
    # simplest: crop randomly some percentage from edges
    crop_w = int(w * (1 - random.uniform(0.05, frac+0.05)))
    crop_h = int(h * (1 - random.uniform(0.05, frac+0.05)))
    crop_w = max(1, min(w, crop_w))
    crop_h = max(1, min(h, crop_h))
    x0 = random.randint(0, w-crop_w)
    y0 = random.randint(0, h-crop_h)
    cropped = pil_img.crop((x0, y0, x0 + crop_w, y0 + crop_h))
    # resize back to original size
    return cropped.resize((w,h), resample=Image.BILINEAR)

def attack_blur(pil_img, kmin=3, kmax=9):
    # gaussian blur via PIL.ImageFilter.GaussianBlur; radius ~ kernel/2
    k = random.choice([3,5,7,9])
    radius = k/2.0
    return pil_img.filter(ImageFilter.GaussianBlur(radius=radius))

def attack_noise(pil_img, var_min=5, var_max=20):
    arr = np.array(pil_img).astype(np.float32)
    var = random.uniform(var_min, var_max)
    sigma = var**0.5
    noise = np.random.normal(0, sigma, arr.shape)
    noisy = arr + noise
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy)

def attack_jpeg(pil_img, qmin=25, qmax=90):
    q = random.randint(qmin, qmax)
    buf = io.BytesIO()
    pil_img.save(buf, format='JPEG', quality=q)
    buf.seek(0)
    return Image.open(buf).convert("RGB")

def attack_combination(pil_img):
    # random combination of 2-3 attacks
    img = pil_img
    choices = [attack_crop, attack_blur, attack_noise, attack_jpeg]
    ops = random.sample(choices, k=random.randint(1,3))
    for op in ops:
        img = op(img)
    return img


In [10]:
# @title (RUN) Already have train/test splits for leave-one-out
# Colab cell 3 (updated): create per-run train/test splits for leave-one-out (skip if already exists)
from pathlib import Path
import random, shutil

ROOT = Path("/content/drive/MyDrive/synthbuster_dataset")
OUT_ROOT = Path("/content/drive/MyDrive/synthbuster_runs")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

ai_gens = [p.name for p in ROOT.iterdir() if p.is_dir() and p.name.lower() != "raise"]
real_dir = ROOT / "raise"
assert real_dir.exists(), "real folder 'raise' not found."

# Settings
TEST_RATIO_REAL = 0.20
TEST_RATIO_NON_HOLDOUT = 0.20
random_seed = 42
random.seed(random_seed)

def is_image_file(p):
    return p.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp"]

def make_leave_one_out_dirs(held_out_gen_name):
    run_name = f"leave_one_out_{held_out_gen_name}"
    run_root = OUT_ROOT / run_name
    train_root = run_root / "train"
    test_root = run_root / "test"

    # ✅ Check if already created (train & test have data)
    if train_root.exists() and test_root.exists():
        num_train = sum(1 for _ in train_root.rglob("*") if is_image_file(_))
        num_test = sum(1 for _ in test_root.rglob("*") if is_image_file(_))
        if num_train > 0 and num_test > 0:
            print(f"✅ Skipping {run_name}: already has train/test splits ({num_train} train, {num_test} test)")
            return run_root, train_root, test_root

    # ❌ Otherwise recreate fresh
    if run_root.exists():
        shutil.rmtree(run_root)
    train_root.mkdir(parents=True)
    test_root.mkdir(parents=True)

    # ---- Real images ----
    real_images = [p for p in real_dir.rglob("*") if is_image_file(p)]
    random.shuffle(real_images)
    n_real_test = max(1, int(len(real_images) * TEST_RATIO_REAL))
    real_test = set(real_images[:n_real_test])
    real_train = set(real_images[n_real_test:])

    (train_root / "raise").mkdir(parents=True, exist_ok=True)
    (test_root  / "raise").mkdir(parents=True, exist_ok=True)
    for p in real_train:
        shutil.copy2(p, train_root / "raise" / p.name)
    for p in real_test:
        shutil.copy2(p, test_root / "raise" / p.name)

    # ---- AI generators ----
    for gen in ai_gens:
        target_train_cls = train_root / "ai"
        target_test_cls = test_root / "ai"
        target_train_cls.mkdir(parents=True, exist_ok=True)
        target_test_cls.mkdir(parents=True, exist_ok=True)

        imgs = [p for p in (ROOT / gen).rglob("*") if is_image_file(p)]
        random.shuffle(imgs)

        if gen == held_out_gen_name:
            for p in imgs:
                shutil.copy2(p, test_root / "ai" / f"{gen}__{p.name}")
        else:
            n_test = int(len(imgs) * TEST_RATIO_NON_HOLDOUT)
            test_part = imgs[:n_test]
            train_part = imgs[n_test:]
            for p in train_part:
                shutil.copy2(p, train_root / "ai" / f"{gen}__{p.name}")
            for p in test_part:
                shutil.copy2(p, test_root / "ai" / f"{gen}__{p.name}")

    num_train = sum(1 for _ in train_root.rglob("*") if is_image_file(_))
    num_test = sum(1 for _ in test_root.rglob("*") if is_image_file(_))
    print(f"✅ Created {run_name}: {num_train} train, {num_test} test")
    return run_root, train_root, test_root

# ---- Run for all AI generators ----
run_roots = {}
for held in ai_gens:
    rr, tr, te = make_leave_one_out_dirs(held)
    run_roots[held] = rr


✅ Skipping leave_one_out_dalle3: already has train/test splits (1040 train, 359 test)
✅ Skipping leave_one_out_firefly: already has train/test splits (1040 train, 359 test)
✅ Skipping leave_one_out_stable-diffusion-xl: already has train/test splits (960 train, 439 test)


In [ ]:
# @title (NOT RUN) Already have train/test splits for leave-one-out
# Colab cell 3: create per-run train/test splits for leave-one-out (one held-out generator)
from pathlib import Path
import random, shutil
ROOT = Path("/content/drive/MyDrive/synthbuster_dataset")
OUT_ROOT = Path("/content/drive/MyDrive/synthbuster_runs")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

ai_gens = [p.name for p in ROOT.iterdir() if p.is_dir() and p.name.lower() != "raise"]
real_dir = ROOT / "raise"
assert real_dir.exists(), "real folder 'raise' not found."

# Settings
TEST_RATIO_REAL = 0.20  # fraction of real images reserved to test for each run
TEST_RATIO_NON_HOLDOUT = 0.20  # for non-held-out AI generators we can still reserve some val images
random_seed = 42
random.seed(random_seed)

def make_leave_one_out_dirs(held_out_gen_name):
    run_name = f"leave_one_out_{held_out_gen_name}"
    run_root = OUT_ROOT / run_name
    if run_root.exists():
        shutil.rmtree(run_root)  # start fresh
    train_root = run_root / "train"
    test_root = run_root / "test"
    train_root.mkdir(parents=True)
    test_root.mkdir(parents=True)

    # 1) Real images -> split into train/test (test real subset goes to test set)
    real_images = [p for p in real_dir.rglob("*") if is_image_file(p)]
    random.shuffle(real_images)
    n_real_test = max(1, int(len(real_images) * TEST_RATIO_REAL))
    real_test = set(real_images[:n_real_test])
    real_train = set(real_images[n_real_test:])

    # copy real
    (train_root / "raise").mkdir(parents=True, exist_ok=True)
    (test_root  / "raise").mkdir(parents=True, exist_ok=True)
    for p in real_train:
        shutil.copy2(p, train_root / "raise" / p.name)
    for p in real_test:
        shutil.copy2(p, test_root / "raise" / p.name)

    # 2) AI generators: for held-out generator -> ALL go to test folder (as AI class)
    for gen in ai_gens:
        gen_dir = ROOT / gen
        target_train_cls = train_root / "ai"  # we combine all AI gens into one 'ai' class for training
        target_test_cls  = test_root  / "ai"
        target_train_cls.mkdir(parents=True, exist_ok=True)
        target_test_cls.mkdir(parents=True, exist_ok=True)

    # Copy AI images: held-out -> test; others -> train (with small val split optionally)
    for gen in ai_gens:
        gen_dir = ROOT / gen
        imgs = [p for p in gen_dir.rglob("*") if is_image_file(p)]
        random.shuffle(imgs)
        if gen == held_out_gen_name:
            # all go to test
            for p in imgs:
                shutil.copy2(p, test_root / "ai" / f"{gen}__{p.name}")
        else:
            # non-heldout: split train/test (we'll mostly use train)
            n_test = int(len(imgs) * TEST_RATIO_NON_HOLDOUT)
            test_part = imgs[:n_test]
            train_part = imgs[n_test:]
            for p in train_part:
                shutil.copy2(p, train_root / "ai" / f"{gen}__{p.name}")
            for p in test_part:
                shutil.copy2(p, test_root / "ai" / f"{gen}__{p.name}")

    # sanity counts
    def count_images(folder):
        return sum(1 for f in folder.rglob("*") if is_image_file(f))
    print(f"Run {run_name}: train size:", count_images(train_root), "test size:", count_images(test_root))
    return run_root, train_root, test_root

# Example: create for all held-out ai generators
run_roots = {}
for held in ai_gens:
    rr, tr, te = make_leave_one_out_dirs(held)
    run_roots[held] = rr


In [3]:
# @title (RUN) Install timm, imports, model factory (MobileViT-V2-100) and helper to adapt classifier
# Colab cell 4: install timm, imports, model loader helper
!pip install -q timm

import timm
import torch, torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, WeightedRandomSampler
import os, copy, math
from collections import Counter
from tqdm import tqdm
from sklearn.metrics import f1_score, balanced_accuracy_score, accuracy_score
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

def create_mobilevitv2_100(num_classes, pretrained=True):
    model = timm.create_model("mobilevitv2_100", pretrained=pretrained)
    # adapt classifier
    try:
        model.reset_classifier(num_classes=num_classes)
    except Exception:
        # fallback
        if hasattr(model, "classifier"):
            in_f = model.classifier.in_features
            model.classifier = nn.Linear(in_f, num_classes)
        elif hasattr(model, "head"):
            in_f = model.head.in_features
            model.head = nn.Linear(in_f, num_classes)
        else:
            raise RuntimeError("Cannot find classifier to replace")
    return model


Device: cuda


In [4]:
# @title (RUN) Train & evaluate function (single run). Uses two-LR, label smoothing, ReduceLROnPlateau, mixed precision
# Colab cell 5: training loop function (one run)
import os
from torch.cuda.amp import GradScaler, autocast

def train_one_run(run_root: Path,
                  image_size=256,
                  batch_size=32,
                  num_workers=2,
                  num_epochs=30,
                  laplacian_enhance=True,
                  save_dir_base="/content/drive/MyDrive/synthbuster_runs_checkpoints"):
    print(f"🚀 Starting training in {run_root} with MobileViTv2_100 ...")
    run_root = Path(run_root)
    train_dir = run_root / "train"
    test_dir  = run_root / "test"
    run_name = run_root.name
    save_dir = Path(save_dir_base) / run_name
    save_dir.mkdir(parents=True, exist_ok=True)
    best_model_path = save_dir / "best_model.pth"
    last_checkpoint_path = save_dir / "last_checkpoint.pth"
    csv_log_path = save_dir / "training_log.csv"

    # transforms
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(image_size, scale=(0.7,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.2,0.2,0.2,0.05),
        transforms.RandomGrayscale(p=0.02),
        transforms.ToTensor(),
        # RandomErasing will be applied in training loop using torchvision.RandomErasing as transform
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
    # we will apply RandomErasing via dedicated transform
    #train_transform.transforms.insert(4, transforms.RandomErasing(p=0.2, scale=(0.02,0.2), ratio=(0.3,3.3)))

    val_transform = transforms.Compose([
        transforms.Resize((image_size,image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])

    # datasets
    train_dataset = datasets.ImageFolder(root=str(train_dir), transform=train_transform)
    val_dataset = datasets.ImageFolder(root=str(test_dir), transform=val_transform)
    classes = train_dataset.classes
    print("Classes:", classes)
    num_classes = len(classes)

    # sampler for class imbalance
    targets = [s[1] for s in train_dataset.samples]
    counts = Counter(targets)
    class_weights = {cls: 1.0 / count for cls, count in counts.items()}
    sample_weights = [class_weights[t] for t in targets]
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler,
                              num_workers=num_workers, pin_memory=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                              num_workers=num_workers, pin_memory=True)

    # model
    model = create_mobilevitv2_100(num_classes=num_classes, pretrained=True)
    model = model.to(device)

    # loss
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

    # two-LR optimizer
    backbone_params, head_params = [], []
    for name, p in model.named_parameters():
        if any(k in name.lower() for k in ("classifier","head")):
            head_params.append(p)
        else:
            backbone_params.append(p)

    optimizer = torch.optim.AdamW([
        {'params': backbone_params, 'lr': 5e-5},
        {'params': head_params, 'lr': 2e-4}
    ], weight_decay=1e-4)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)
    scaler = GradScaler() if torch.cuda.is_available() else None

    # training bookkeeping
    best_val_acc = 0.0
    best_wts = copy.deepcopy(model.state_dict())
    history = {"epoch":[],"train_loss":[],"train_acc":[],"val_loss":[],"val_acc":[],"val_macro_f1":[],"val_bal_acc":[]}

    early_stopping_patience = 6
    no_improve = 0
    start_epoch = 0

    # resume if exists
    if last_checkpoint_path.exists():
        print("Resuming from checkpoint:", last_checkpoint_path)
        ck = torch.load(last_checkpoint_path, map_location=device, weights_only=False)
        model.load_state_dict(ck["model_state"])
        optimizer.load_state_dict(ck["optimizer_state"])
        history = ck["history"]
        start_epoch = ck["epoch"]
        best_val_acc = ck.get("best_val_acc", 0.0)
        print(f"Resumed at epoch {start_epoch}, best_val_acc {best_val_acc:.4f}")

    for epoch in range(start_epoch, num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")
        for images, labels in loop:
            # apply optional laplacian enhancement per-batch (on PIL would be earlier; here images are tensors)
            # easier approach: apply laplacian on original files by using a custom dataset; we opt for simpler: skip per-batch.
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            if scaler is not None:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            running_loss += loss.item() * images.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += images.size(0)
            loop.set_postfix(loss=running_loss/total, acc=100.*correct/total)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = 100. * correct / total

        # validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                preds = outputs.argmax(dim=1)
                val_correct += (preds == labels).sum().item()
                val_total += images.size(0)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = 100. * val_correct / val_total
        val_macro_f1 = f1_score(all_labels, all_preds, average='macro')
        val_bal_acc = balanced_accuracy_score(all_labels, all_preds)

        scheduler.step(val_acc)

        history["epoch"].append(epoch+1)
        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)
        history["val_macro_f1"].append(val_macro_f1)
        history["val_bal_acc"].append(val_bal_acc)

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}% | Macro-F1: {val_macro_f1:.4f}, Bal Acc: {val_bal_acc:.4f}")

        # save best
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), str(best_model_path))
            print("Saved best model:", best_model_path)
            no_improve = 0
        else:
            no_improve += 1
            print(f"No improvement for {no_improve} epochs.")

        # early stop
        if no_improve >= early_stopping_patience:
            print("Early stopping triggered.")
            break

        # save checkpoint
        torch.save({
            "epoch": epoch+1,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "history": history,
            "best_val_acc": best_val_acc
        }, str(last_checkpoint_path))

        # write csv
        pd.DataFrame(history).to_csv(csv_log_path, index=False)

    # load best weights
    model.load_state_dict(best_wts)
    print("Training complete. Best val acc:", best_val_acc)
    return {
        "model": model,
        "history": history,
        "save_dir": save_dir,
        "val_acc": best_val_acc,
        "classes": classes,
        "val_loader": val_loader,
        "test_dir": test_dir
    }


In [ ]:
# @title (NOT RUN)
# Now actually call the function
result = train_one_run(
    run_root="/content/drive/MyDrive/synthbuster_runs/leave_one_out_dalle3",
    image_size=256,
    batch_size=32,
    num_workers=2,
    num_epochs=30,
    laplacian_enhance=True,
    save_dir_base="/content/drive/MyDrive/synthbuster_runs_checkpoints"
)


🚀 Starting training in /content/drive/MyDrive/synthbuster_runs/leave_one_out_dalle3 with MobileViTv2_100 ...
Classes: ['ai', 'raise']
Resuming from checkpoint: /content/drive/MyDrive/synthbuster_runs_checkpoints/leave_one_out_dalle3/last_checkpoint.pth
Resumed at epoch 13, best_val_acc 92.7577


Epoch 14/30 [Train]:   0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 14/30 [Train]: 100%|██████████| 33/33 [18:10<00:00, 33.04s/it, acc=99.1, loss=0.235]


Epoch 14/30 | Train Loss: 0.2353, Train Acc: 99.13% | Val Loss: 0.3681, Val Acc: 89.69% | Macro-F1: 0.8929, Bal Acc: 0.8862
No improvement for 1 epochs.


Epoch 15/30 [Train]:   0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 15/30 [Train]: 100%|██████████| 33/33 [17:44<00:00, 32.26s/it, acc=99, loss=0.232]


Epoch 15/30 | Train Loss: 0.2325, Train Acc: 99.04% | Val Loss: 0.3518, Val Acc: 91.92% | Macro-F1: 0.9167, Bal Acc: 0.9112
No improvement for 2 epochs.


...

Epoch 17/30 | Train Loss: 0.2337, Train Acc: 99.23% | Val Loss: 0.3478, Val Acc: 91.64% | Macro-F1: 0.9138, Bal Acc: 0.9081
No improvement for 4 epochs.


Epoch 18/30 [Train]:   0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 18/30 [Train]: 100%|██████████| 33/33 [17:43<00:00, 32.24s/it, acc=99.7, loss=0.226]


Epoch 18/30 | Train Loss: 0.2260, Train Acc: 99.71% | Val Loss: 0.3519, Val Acc: 92.48% | Macro-F1: 0.9225, Bal Acc: 0.9168
No improvement for 5 epochs.


Epoch 19/30 [Train]:   0%|          | 0/33 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Epoch 19/30 [Train]: 100%|██████████| 33/33 [18:09<00:00, 33.01s/it, acc=99.6, loss=0.224]


Epoch 19/30 | Train Loss: 0.2243, Train Acc: 99.62% | Val Loss: 0.3495, Val Acc: 92.20% | Macro-F1: 0.9197, Bal Acc: 0.9143
No improvement for 6 epochs.
Early stopping triggered.
Training complete. Best val acc: 92.75766016713092


In [ ]:
# @title (RUN) “leave out” one generator entirely from training and use it as unseen test data for Cross-Generator Generalization
# Colab cell 6: run training for each held-out AI generator
from pathlib import Path

RUNS_DIR = Path("/content/drive/MyDrive/synthbuster_runs")
CHECKPOINTS_DIR = "/content/drive/MyDrive/synthbuster_runs/synthbuster_runs_checkpoints"

results = {}
for held in ai_gens:
    print("==== Starting run — held-out:", held, "====")
    run_root = RUNS_DIR / f"leave_one_out_{held}"
    out = train_one_run(run_root,
                        image_size=256,
                        batch_size=32,
                        num_workers=4,
                        num_epochs=40,
                        laplacian_enhance=False,  # set True to apply Laplacian preprocessing (requires custom dataset)
                        save_dir_base=CHECKPOINTS_DIR)
    results[held] = out
    print("==== Finished run —", held, "best val acc:", out['val_acc'])


==== Starting run — held-out: dalle3 ====
🚀 Starting training in /content/drive/MyDrive/synthbuster_runs/leave_one_out_dalle3 with MobileViTv2_100 ...
Classes: ['ai', 'raise']


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

/tmp/ipython-input-3510562111.py:83: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if torch.cuda.is_available() else None


Resuming from checkpoint: /content/drive/MyDrive/synthbuster_runs/synthbuster_runs_checkpoints/leave_one_out_dalle3/last_checkpoint.pth
Resumed at epoch 12, best_val_acc 92.7577


...

Epoch 12/40 [Train]:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-3510562111.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 12/40 [Train]: 100%|██████████| 30/30 [05:31<00:00, 11.06s/it, acc=99.6, loss=0.231]


Epoch 12/40 | Train Loss: 0.2306, Train Acc: 99.58% | Val Loss: 0.5446, Val Acc: 76.99% | Macro-F1: 0.7671, Bal Acc: 0.7870
No improvement for 2 epochs.


Epoch 13/40 [Train]:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-3510562111.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 13/40 [Train]: 100%|██████████| 30/30 [05:32<00:00, 11.08s/it, acc=99, loss=0.239]


Epoch 13/40 | Train Loss: 0.2386, Train Acc: 98.96% | Val Loss: 0.5539, Val Acc: 76.54% | Macro-F1: 0.7623, Bal Acc: 0.7828
No improvement for 3 epochs.


Epoch 14/40 [Train]:   0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/tmp/ipython-input-3510562111.py:117: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Epoch 14/40 [Train]:  27%|██▋       | 8/30 [01:32<02:47,  7.64s/it, acc=100, loss=0.23]